In [6]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [7]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-08 04:55:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.74MB/s    in 0.2s    

2021-02-08 04:55:52 (4.74 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [9]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [10]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df = vine_df.dropna()
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
| RA51CP8TR5A2L|          5|            0|          0|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|                N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
| RX27XIIWY5JPB|          4|            0|          0|   N|     

In [11]:
# 1 Filter the data and create a new DataFrame or table to retrieve all the rows 
# where the total_votes count is equal to or greater than 20 to pick reviews that
# are more likely to be helpful and to avoid having division by zero errors later on.

total_votes_df = vine_df.filter("total_votes>=20")
total_votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|     

In [12]:
# 2 Filter the new DataFrame or table created in Step 1 and create a new DataFrame 
# or table to retrieve all the rows where the number of helpful_votes divided by total_votes 
# is equal to or greater than 50%

helpful_votes_df = vine_df.filter("helpful_votes/total_votes>=.50")
helpful_votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
|R15C7QEZT0LGZN|          4|            2|          2|   N|                Y|
| RWASY7FKI7QOT|          5|            2|          3|   N|                Y|
|R22H4FGVD5O52O|          4|            1|          1|   N|                Y|
| RJ9HWWMU4IAHF|          4|            1|          1|   N|                Y|
|R3629T8HDV5VWU|          5|            1|          1|   N|                Y|
|R2QGEJRU4ENYN2|          5|            1|          1|   N|                N|
|R1HQZDBB90LMFH|          5|            2|          2|   N|     

In [13]:
# 3 Filter the DataFrame or table created in Step 2, and create a new DataFrame or 
# table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

paid_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
paid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R6EMI5RKW73N3|          4|            1|          2|   Y|                N|
| R2IZD1134PGIP|          4|            1|          2|   Y|                N|
|R2IVDAK7IYLXJS|          4|            1|          1|   Y|                N|
| RCUJSPL51V519|          4|            1|          1|   Y|                N|
| R3ZNTDTGSY8SC|          4|            1|          1|   Y|                N|
|R2DC9TNKKI8EXN|          5|            1|          1|   Y|                N|
|R2GX9CYB2H258C|          5|            2|          2|   Y|                N|
| R749PGSBA9HP6|          4|            1|          1|   Y|                N|
|R170PR0NEEIW2E|          4|            2|          2|   Y|                N|
| RMAG0OPUODKQC|          5|            4|          4|   Y|     

In [14]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the 
#Vine program (unpaid), vine == 'N'.

unpaid_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
unpaid_review_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
|R15C7QEZT0LGZN|          4|            2|          2|   N|                Y|
| RWASY7FKI7QOT|          5|            2|          3|   N|                Y|
|R22H4FGVD5O52O|          4|            1|          1|   N|                Y|
| RJ9HWWMU4IAHF|          4|            1|          1|   N|                Y|
|R3629T8HDV5VWU|          5|            1|          1|   N|                Y|
|R2QGEJRU4ENYN2|          5|            1|          1|   N|                N|
|R1HQZDBB90LMFH|          5|            2|          2|   N|     

In [15]:
# Determine the total number of reviews (paid and unpaid).

print("Total number of Paid Reviews: %f" %paid_review_df.count())
print("Total number of Unpaid Reviews: %f" %unpaid_review_df.count())


Total number of Paid Reviews: 945.000000
Total number of Unpaid Reviews: 296087.000000


In [26]:
# Determine the number of 5-star reviews (paid vs unpaid).

paid_review_5star_df = paid_review_df.filter(paid_review_df["star_rating"] == "5")
print("5-star Paid Reviews: %f" % paid_review_5star_df.count())

unpaid_review_5star_df = unpaid_review_df.filter(unpaid_review_df["star_rating"] == "5")
print("5-star Unpaid Reviews: %f" % unpaid_review_5star_df.count())

5-star Paid Reviews: 332.000000
5-star Unpaid Reviews: 169777.000000


In [31]:
# Determine the percentage of 5-star reviews (paid vs unpaid).

percent_paid_5star = (paid_review_5star_df.count()/helpful_votes_df.count())*100
print("Paid percentage of 5-star reviews: %f" % percent_paid_5star)
percent_unpaid_5star  = (unpaid_review_5star_df.count()/helpful_votes_df.count())*100
print("Unpaid percentage of 5-star reviews: %f" % percent_unpaid_5star)

Paid percentage of 5-star reviews: 0.111772
Unpaid percentage of 5-star reviews: 57.157815
